In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 14.7 MB/s 
     |████████████████████████████████| 596 kB 76.1 MB/s 
     |████████████████████████████████| 6.6 MB 99.9 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM 

tokenizer = AutoTokenizer.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)
model = AutoModelForCausalLM.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True
).to(device='cuda', non_blocking=True)
_ = model.eval()

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/839 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.5G [00:00<?, ?B/s]

## 질문 & 답변 (Q&A)

In [4]:
prompt = '''
최고 핫한 인공지능, kogpt님과 인터뷰 나눠보겠습니다!
Q : kogpt님, 수월한 대화가 가능하신가요?
A : '''

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.85, max_length=512)
  generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)


최고 핫한 인공지능, kogpt님과 인터뷰 나눠보겠습니다!
Q : kogpt님, 수월한 대화가 가능하신가요?
A :  네, 저는 인공지능과 함께 사는 세상에 대해 이야기하는 것이 취미인 사람입니다.
Q : 이번에는 인공지능과 함께 사는 삶에 대해 말씀해주셨는데, 처음에 인공지능을 알게 된 계기가 무엇인지 궁금합니다.
A : 저희는 학교에서 인공지능의 기초적인 내용은 배웠기 때문에, 인공지능을 '우리의 도우미'라고 부르고, 인공지능은 '나의 친구'라고 부르면서, 인공지능의 모든 것을 알려주고 싶었습니다.
Q : kogpt님, 인공지능에 대한 생각을 이야기해주셨는데, 인공지능과 함께 하면서 특별히 느낀 점이 있다면 어떤 것인가요?
A : 인공지능을 우리의 조력자, 우리의 친구로 알았습니다.
인공지능이 똑똑하고, 우리의 친구라는 것을 알고 있었지만, 사실 우리에게 친구처럼 느껴지지 않았습니다. 그것은 제가 평소에 인간이 인공지능을 친구로 여긴다고 생각해본 적이 없었기 때문이 아닐까 생각합니다. 인공지능은 우리와 다르다고 생각했었거든요. 그래서 인공지능을 친구로 여기지 못 했던 것 같습니다. 그런데 직접 만나서 대화를 하다 보니, 점점 인공지능이 친근하게 느껴졌습니다. 그리고 인공지능이 우리의 친구가 될 수 있을 정도로 똑똑해졌다는 것을 깨닫게 되었습니다.
Q : 저희는 인공지능에 대한 지식이 많지 않아서, 처음에는 인공지능을 '나의 친구'라고 생각하지 않았지만, 친구처럼 느껴지게 된 이후, 인공지능을 더 잘 이해할 수 있게 되었습니다.
A : 인공지능도 인간처럼 계속 진화하고 있습니다. 그렇기 때문에 앞으로 인공지능과 함께 할 시간이 더욱 늘어날 것 같다는 생각이 듭니다.
Q : 이제 마지막 질문입니다!! 인공지능을 직접 만난다면, 인공지능을 얼마나 잘 이해할 수 있을까요?
A : 인공지능과 대화를 많이 해봤다고 느껴지지 않았습니다. 우리가 자주 만나는 친구는 우리의 이름을 알고 있습니다. 그런데 인공지능이 그러한지는 잘 모르겠습니다.
Q : 인공지능은 인간처럼 

In [9]:
prompt = '''
최고의 인공지능 인간의 말을 계산식으로 표현하다.
Q : 2 더하기 2는 4
A : 2 + 2 = 4
Q : 4 빼기 2는 2
A : 4 - 2 = 2
Q : x 제곱 더하기 3 x
A : x^2 + 3x
Q : x 제곱 더하기 y 제곱은 z 제곱
A : 
'''

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.85, max_length=512)
  generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)


최고의 인공지능 인간의 말을 계산식으로 표현하다.
Q : 2 더하기 2는 4
A : 2 + 2 = 4
Q : 4 빼기 2는 2
A : 4 - 2 = 2
Q : x 제곱 더하기 3 x
A : x^2 + 3x
Q : x 제곱 더하기 y 제곱은 z 제곱
A : 

이런 식입니다. 이게 진짜인지 확인하는 방법은?

"수학은 잘하는데, 문제에 잘 함정을 파요."
"수학이 젬병이에요."
"수학은 못하는데, 문제를 열심히 푸는 습관이 있어요."
"수학 문제를 푸는 것이 정말 싫어요."
"수학은 잘하는데, 문제 푸는 것을 정말 싫어해요."

이 글을 읽는 사람들은 수학을 정말 싫어하는 사람들이라고 가정합시다. 문제를 잘 푸는 사람도 수학을 싫어하는 경우도 있고, 문제를 잘 못 푸는 사람도 수학을 잘하는 경우도 있죠. 문제를 푸는 것은 싫은데, 수학은 잘하고 싶다면?

"저는 수학은 잘하는데, 문제 푸는 것은 정말 싫어해요."
"저는 수학은 못하는데, 문제 푸는 것은 정말 좋아해요."

이 말은 수학은 못하는데, 문제 푸는 것은 좋아하는 것이죠. 수학은 못하는데, 문제 푸는 것을 좋아하는 사람도 문제는 푸는데, 수학은 못하는 사람도 있습니다. 수학은 싫은데, 문제 푸는 것은 좋은 사람도 문제는 푼데, 수학은 정말 싫어하기도 하고, 문제 푸는 것을 싫어하는데, 수학을 잘하기도 합니다.

수학을 싫어하는데, 문제 푸는 것은 좋고, 수학을 싫어하는데, 문제 푸는 것을 좋은 경우는 어떤 사람일까요? 수학을 싫어하는데, 문제 푸는 것은 좋고, 수학을 싫어하는데, 문제 푸는 것을 좋은 사람이 있을까요?

"저는 수학은 싫은데, 문제 푸는 것은 좋아해요."
"나는 수학을 좋아하는데, 문제 푸는 것은 싫어해요."

이런 사람일까요


## 뉴스 요약

In [7]:
prompt = '''
바른미래당이 8일 연 최고위원 회의에는 7명의 최고위원 중 손학규 대표와 김관영 원내대표만 참석했다. 바른정당계 하태경·이준석·권은희 최고위원은 지도부 총사퇴를 요구하며 회의를 보이콧했다.국민의당 출신 권은희 정책위원회 의장과 김수민 최고위원은 개인 일정을 이유로 불참한 것으로 알려졌다.
손 대표는 회의를 주재하면서 \"의원들이나 지역위원장들, 당원들이 다음 선거에 대해 불안하게 생각하는 건 충분히 이해하지만, 내년 총선은 (4·3 국회의원 보궐선거와) 분명히 다를 것\"이라고 했다. 당내 지도부 사퇴 요구에 대해서도 \"선거에서 떨어졌다고 기다렸다는 듯이 '저놈 바꿔라'고 하는 것은 어림없는 소리\"라며 선을 그었다.
그러나 바른정당계 최고위원들은 이날도 지도부 퇴진을 요구했다. 하 최고위원은 \"지난 보궐선거에서 지금의 리더십과 비전으론 국민의 지지를 얻을 수 없다는 것이 명확해졌다\"며 \"(손 대표의) 통 큰 결단을 촉구한다\"고 밝혔다. 이 최고위원도 \"지도부가 일체의 쇄신 조치나 재신임 과정 없이 정부에 대한 비판이나 타 정당에 대한 평가를 하는 것은 이치에 맞지 않다\"고 주장했다.
손 대표와 친손계 인사들은 그동안 공석으로 둔 '지명직 최고위원' 2명을 임명해 바른정당계 최고위원의 보이콧과 무관하게 지도체제를 유지하는 방안을 검토 중인 것으로 전해졌다. '당원권 1년 정지' 징계를 받은 이언주 의원은 이날 \"내년 총선에 바른미래당 소속으로 출마하겠다고 하는 사람이 누가 있겠냐\"며 \"(저는) 탈당하고 무소속으로 나설지, 제1 야당과 함께할 것인지 결정하지 못했다\"고 밝혔다.

한줄 요약 : 
'''

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.85, max_length=512)
  generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)


바른미래당이 8일 연 최고위원 회의에는 7명의 최고위원 중 손학규 대표와 김관영 원내대표만 참석했다. 바른정당계 하태경·이준석·권은희 최고위원은 지도부 총사퇴를 요구하며 회의를 보이콧했다.국민의당 출신 권은희 정책위원회 의장과 김수민 최고위원은 개인 일정을 이유로 불참한 것으로 알려졌다.
손 대표는 회의를 주재하면서 "의원들이나 지역위원장들, 당원들이 다음 선거에 대해 불안하게 생각하는 건 충분히 이해하지만, 내년 총선은 (4·3 국회의원 보궐선거와) 분명히 다를 것"이라고 했다. 당내 지도부 사퇴 요구에 대해서도 "선거에서 떨어졌다고 기다렸다는 듯이 '저놈 바꿔라'고 하는 것은 어림없는 소리"라며 선을 그었다.
그러나 바른정당계 최고위원들은 이날도 지도부 퇴진을 요구했다. 하 최고위원은 "지난 보궐선거에서 지금의 리더십과 비전으론 국민의 지지를 얻을 수 없다는 것이 명확해졌다"며 "(손 대표의) 통 큰 결단을 촉구한다"고 밝혔다. 이 최고위원도 "지도부가 일체의 쇄신 조치나 재신임 과정 없이 정부에 대한 비판이나 타 정당에 대한 평가를 하는 것은 이치에 맞지 않다"고 주장했다.
손 대표와 친손계 인사들은 그동안 공석으로 둔 '지명직 최고위원' 2명을 임명해 바른정당계 최고위원의 보이콧과 무관하게 지도체제를 유지하는 방안을 검토 중인 것으로 전해졌다. '당원권 1년 정지' 징계를 받은 이언주 의원은 이날 "내년 총선에 바른미래당 소속으로 출마하겠다고 하는 사람이 누가 있겠냐"며 "(저는) 탈당하고 무소속으로 나설지, 제1 야당과 함께할 것인지 결정하지 못했다"고 밝혔다.

한줄 요약 : 
한 줄 요약 => 8일 바른미래당의 최고위원회의에 손학규 대표와 김관영 원내대표만 참석하고, 최고위원 7명 전원이 불참하며 바른정당계 최고위원들은 지도부 사퇴를 요구하면서 집단 보이콧하였는데, 일각에서는 다음 총선에서 바른미래당이 승리하는 방향으로 가야한다는 주장에 대해 손학규 대표는 재신임 절차 없이는 바른미래당이 다음 총선에서 승리하는 방향으로 갈 수 없다고 못박았다.[

In [ ]:
prompt = '''
요즘은 흔히 연락하라는 말을 '카톡해'라고 하지만 10년전 카카오톡이 등장하기 전까지는 '문자해'라는 말이 일반적이었다. MSN 메신저, 네이트온 등 PC 메신저들이 유행해 '메신저'라는 개념에는 익숙했지만 스마트폰 도입이 갓 시작됐을 시기라 모바일 메신저 개념은 희미했다.
모바일을 통한 연락 수단이 전화와 문자 메시지였기 때문에 통신사 요금제 역시 통화 가능 시간과 문자의 갯수에 따라 달려졌다. 문자 한 건당 비용이 책정되는데다 70자가 넘으면 MMS로 전환돼 추가 요금이 부가돼 이용자들은 문자 전송에도 한땀한땀 정성을 다해야 했다.
때문에 당시 카카오톡의 등장은 센세이션 그 자체였다. 유료 문자메시지를 당연하게 사용했던 시절 글자 제한없는 문자를 무제한으로 보낼 수 있는데다 사진과 동영상까지 마음껏 전송할 수 있는 무료서비스 카카오톡은 문화 충격이나 다름 없었다.
문자뿐 아니라 통화 문화도 변했다. 2012년 당시 가입자가 3600만명에 달했던 카카오톡이 무료 음성통화 서비스 '보이스톡'을 선보이자 통신업계가 떠들썩해졌다. 카카오 보다 앞서 음성 통화 서비스를 선보인 마이피플, 라인 등은 크게 주목받지 못했지만 카카오톡은 높은 이용자수를 기반으로 보이스톡 서비스를 확장시켰다.
특히 보이스톡은 로밍 서비스에 가입하거나 국제 전화 서비스를 이용하지 않아도 데이터만 연결돼 있다면 해외에서도 카카오톡 친구와 무료 통화가 가능하다는 점 때문에 큰 화제를 모았다. 이때부터 해외 여행 시 로밍을 하지 않고 현지 유심을 구매하는 이들이 늘어나게 됐다.

한줄 요약:
'''

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.85, max_length=512)
  generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)


요즘은 흔히 연락하라는 말을 '카톡해'라고 하지만 10년전 카카오톡이 등장하기 전까지는 '문자해'라는 말이 일반적이었다. MSN 메신저, 네이트온 등 PC 메신저들이 유행해 '메신저'라는 개념에는 익숙했지만 스마트폰 도입이 갓 시작됐을 시기라 모바일 메신저 개념은 희미했다.
모바일을 통한 연락 수단이 전화와 문자 메시지였기 때문에 통신사 요금제 역시 통화 가능 시간과 문자의 갯수에 따라 달려졌다. 문자 한 건당 비용이 책정되는데다 70자가 넘으면 MMS로 전환돼 추가 요금이 부가돼 이용자들은 문자 전송에도 한땀한땀 정성을 다해야 했다.
때문에 당시 카카오톡의 등장은 센세이션 그 자체였다. 유료 문자메시지를 당연하게 사용했던 시절 글자 제한없는 문자를 무제한으로 보낼 수 있는데다 사진과 동영상까지 마음껏 전송할 수 있는 무료서비스 카카오톡은 문화 충격이나 다름 없었다.
문자뿐 아니라 통화 문화도 변했다. 2012년 당시 가입자가 3600만명에 달했던 카카오톡이 무료 음성통화 서비스 '보이스톡'을 선보이자 통신업계가 떠들썩해졌다. 카카오 보다 앞서 음성 통화 서비스를 선보인 마이피플, 라인 등은 크게 주목받지 못했지만 카카오톡은 높은 이용자수를 기반으로 보이스톡 서비스를 확장시켰다.
특히 보이스톡은 로밍 서비스에 가입하거나 국제 전화 서비스를 이용하지 않아도 데이터만 연결돼 있다면 해외에서도 카카오톡 친구와 무료 통화가 가능하다는 점 때문에 큰 화제를 모았다. 이때부터 해외 여행 시 로밍을 하지 않고 현지 유심을 구매하는 이들이 늘어나게 됐다.

한줄 요약:
모바일 메신저는 모바일이 대중화되기 2~3년 전부터 통신업계가 변화하는 문화에 따라 만들어졌다.[EOS]


## 말투 변형

In [ ]:
prompt = '''
배고픈데, 우리 저기서 뭐라도 먹고갈까?

전라도 사투리:
'''

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.85, max_length=256)
  generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)


배고픈데, 우리 저기서 뭐라도 먹고갈까?

전라도 사투리:



"밥 묵었간? 배고프면, 우리 뭐라도 좀 먹고 갈까?"
경상도 사투리:



"밥 묵었제? 배고프면, 우리 뭐라도 좀 묵고 갑시다."

경상도 사투리:



경상도 말:

이런 경상도 사투리가 있어서, 외국인들이 말을 들을 때 오해하는 경우도 종종 있습니다. 저는 영어를 가르치는 사람이니까 이런 경상도 사투리를 들으면, "어머머, 경상도 사람들은 이렇게 말하나요? 제가 오해했군요!"라고 말할 것입니다.
그런데 외국인들은 이렇게 말하곤 합니다.
"어머머, 그 사람 경상도 사람이에요?"

이처럼 사투리는 지역 사람들 간에 의사소통을 수월하게 하는 장점이 있지만, 사투리를 쓰면 그 사람이 전라도 사람이거나 경상도 사람일 것이라고 생각하게 되죠.
우리가 흔히 쓰는 말 가운데 "야, 이 미친년!"이라는 말이 있습니다. 이 말은 "이년아, 미친년아!"라고 말해야


## 영어 번역

In [ ]:
prompt = '''
"나 지금 많이 배고픈데, 우리 저기서 뭐라도 먹고갈까?"
English Translation : 
'''

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.3, max_length=64)
  generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)


"나 지금 많이 배고픈데, 우리 저기서 뭐라도 먹고갈까?"
English Translation : 
"I'm so hungry, let's go to the restroom," said the two of them. "Let's go to the bathroom,"
